In [1]:
import time                                     #to use time.sleep()
from seleniumwire import webdriver              # using seleniumwire to allow inspect network requests/responses 
from seleniumwire.webdriver import ActionChains # using ActionChains to trigger .click() event on any kind of element

In [2]:
import json
from requests import Request, Session

In [3]:
#Post search variables
#https://www.instagram.com/p/B-pln9YJ_6z/
#https://www.instagram.com/p/Bfkx-6lHbf7/

postUrl = 'https://www.instagram.com/p/B-pln9YJ_6z/'

postUrl

'https://www.instagram.com/p/B-pln9YJ_6z/'

In [4]:
def urlfier (arr):
    return str(''.join(arr))

urlfier(['Its ', 'working ', 'perfectly ', '!'])

'Its working perfectly !'

In [5]:
def queryExtractor(url):
    return url[url.find('after%22%3A%22') + len('after%22%3A%22') : url.find('%3D%3D%22%7D')]

#queryExtractor(requestUrl)

In [ ]:
def getNextHash(response):
    return 

In [6]:
#get the post hash
postHash = postUrl[28:len(postUrl) - 1]
postHash

'B-pln9YJ_6z'

In [7]:
#static variables
queryHash = 'https://www.instagram.com/graphql/query/?query_hash='
matches = [postHash, queryHash, 'shortcode', 'first', 'after']
total_comments = 0

In [8]:
#using ChromeDriver
print('Wait browser open in post page !')
driver = webdriver.Chrome('./chromedriver.exe')
driver.get(postUrl)
print('Ready to go !')

Wait browser open in post page !
Ready to go !


In [9]:
#class of 'load more' button -> glyphsSpriteCircle_add__outline__24__grey_9 u-__7 (detected manually)
#handling the element based on his class
element = driver.find_element_by_class_name('glyphsSpriteCircle_add__outline__24__grey_9.u-__7')
element

<selenium.webdriver.remote.webelement.WebElement (session="5242d8f1842cf7681c6bbcec840fee65", element="73657863-fe3b-428c-ac5e-6b96507f9bba")>

In [10]:
# trigger the button to make next page request
element.click()

In [11]:
#handle the url of Instagram GraphQL comments endpoint 
requestUrl = ''

#search the endpoint url
for request in driver.requests:
    if all(x in request.url for x in matches):
        requestUrl = request.url

requestUrl

'https://www.instagram.com/graphql/query/?query_hash=bc3296d1ce80a24b1b6e40b1e72903f5&variables=%7B%22shortcode%22%3A%22B-pln9YJ_6z%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFDdmRVNS16UW1TMkNFVDJpN1dDbmxLUHpkQVJWZlBQZzNsakJ2UDJfVjRlMGU4MFhnVmRKVm1FN182b1dPTUlkRW42NktacUJjdl9fcGx2aVd3UUVRdw%3D%3D%22%7D'

In [12]:
#handle all hash of all pages
pagesHash = []
pagesHash.append(queryExtractor(requestUrl))

pagesHash

['QVFDdmRVNS16UW1TMkNFVDJpN1dDbmxLUHpkQVJWZlBQZzNsakJ2UDJfVjRlMGU4MFhnVmRKVm1FN182b1dPTUlkRW42NktacUJjdl9fcGx2aVd3UUVRdw']

In [13]:
#Settin Up the custom request infos

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Host": "www.instagram.com",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
}

headers

{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
 'Host': 'www.instagram.com',
 'Sec-Fetch-Dest': 'document',
 'Sec-Fetch-Mode': 'navigate'}

In [14]:
URL = [
        queryHash,
        requestUrl[requestUrl.find('?query_hash=') + len('?query_hash=') : 158 + len('after%22%3A%22')], # variables
        pagesHash[-1], # pagination hash idx 2
        '%3D%3D%22%7D',
      ]

urlfier(URL)

'https://www.instagram.com/graphql/query/?query_hash=bc3296d1ce80a24b1b6e40b1e72903f5&variables=%7B%22shortcode%22%3A%22B-pln9YJ_6z%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFDdmRVNS16UW1TMkNFVDJpN1dDbmxLUHpkQVJWZlBQZzNsakJ2UDJfVjRlMGU4MFhnVmRKVm1FN182b1dPTUlkRW42NktacUJjdl9fcGx2aVd3UUVRdw%3D%3D%22%7D'

In [15]:
# Prepare the first request

prepped = Request('GET',
                  urlfier(URL),
                  headers=headers,
                 ).prepare()

prepped

<PreparedRequest [GET]>

In [16]:
# Request Handler
reqHandler = []

# Edges Handler
#edgesHandler = []

reqHandler#, edgesHandler

[]

In [17]:
# Session Setup
s = Session() 

# Making the request
resp = s.send(prepped) 

reqHandler.append(resp.json())

resp, len(reqHandler)

(<Response [200]>, 1)

In [18]:
total_comments = reqHandler[0]['data']['shortcode_media']['edge_media_to_parent_comment']['count']
total_comments

1503

In [19]:
# Setup some variables

# Get if Has Next Page 
has_next = reqHandler[-1]['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['has_next_page']

# Get the Next Page Hash
next_hash = reqHandler[-1]['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['end_cursor'][:(-2)]

# Append second hash in list
pagesHash.append(next_hash)

has_next, next_hash, pagesHash

(True,
 'QVFDSFNQeW1WYVlfT2ZCZGpMaFhKRS1nLV9mM2xKTVVVRWdTNUY5ZklVcjYtY2FUbTQxb3h1VlJLbzJoUTl3Q18tX3VNd1dXQlhtajVETGNCd1VCWnFPUQ',
 ['QVFDdmRVNS16UW1TMkNFVDJpN1dDbmxLUHpkQVJWZlBQZzNsakJ2UDJfVjRlMGU4MFhnVmRKVm1FN182b1dPTUlkRW42NktacUJjdl9fcGx2aVd3UUVRdw',
  'QVFDSFNQeW1WYVlfT2ZCZGpMaFhKRS1nLV9mM2xKTVVVRWdTNUY5ZklVcjYtY2FUbTQxb3h1VlJLbzJoUTl3Q18tX3VNd1dXQlhtajVETGNCd1VCWnFPUQ'])

In [20]:
req_count = 1
while has_next:
    
    URL[2] = next_hash
    prepped.url = urlfier(URL)
    
    resp = s.send(prepped)
    print(f'{req_count} requests made.')
    req_count += 1
    
    reqHandler.append(resp.json())
    
    has_next = reqHandler[-1]['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['has_next_page']
    next_hash = reqHandler[-1]['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['end_cursor'][:(-2)]
    time.sleep(1.5)

len(reqHandler)

1 requests made.
2 requests made.
3 requests made.
4 requests made.
5 requests made.
6 requests made.
7 requests made.
8 requests made.
9 requests made.
10 requests made.
11 requests made.
12 requests made.
13 requests made.
14 requests made.
15 requests made.
16 requests made.
17 requests made.
18 requests made.
19 requests made.
20 requests made.
21 requests made.
22 requests made.
23 requests made.
24 requests made.
25 requests made.
26 requests made.
27 requests made.
28 requests made.
29 requests made.
30 requests made.
31 requests made.
32 requests made.
33 requests made.
34 requests made.
35 requests made.
36 requests made.
37 requests made.
38 requests made.
39 requests made.
40 requests made.
41 requests made.
42 requests made.
43 requests made.
44 requests made.
45 requests made.
46 requests made.
47 requests made.
48 requests made.
49 requests made.
50 requests made.
51 requests made.
52 requests made.
53 requests made.
54 requests made.
55 requests made.
56 requests made.
5

TypeError: 'NoneType' object is not subscriptable

In [22]:
has_next

False

In [66]:
res_handler = reqHandler

len(res_handler) == len(reqHandler)

True

In [67]:
edges_handler = []

In [76]:
 res_handler[0]['data']['shortcode_media']['edge_media_to_parent_comment']['edges']

[{'node': {'id': '18145711345009751',
   'text': 'That show was fire bro fire. Inspiring',
   'created_at': 1591740362,
   'did_report_as_spam': False,
   'owner': {'id': '5870022855',
    'is_verified': False,
    'profile_pic_url': 'https://scontent-gru2-2.cdninstagram.com/v/t51.2885-19/s150x150/125306034_773308609884821_5132705361019659910_n.jpg?_nc_ht=scontent-gru2-2.cdninstagram.com&_nc_ohc=m2uMxWBMWUgAX_HdI1C&tp=1&oh=022636521a78c68727dc841e7540d8b2&oe=60085CCF',
    'username': 'dust_py'},
   'viewer_has_liked': False,
   'edge_liked_by': {'count': 0},
   'is_restricted_pending': False,
   'edge_threaded_comments': {'count': 0,
    'page_info': {'has_next_page': False, 'end_cursor': None},
    'edges': []}}},
 {'node': {'id': '17870643427785058',
   'text': 'I need the next season NOW😖 This show was so good. But props to everyone who worked on this!',
   'created_at': 1591761903,
   'did_report_as_spam': False,
   'owner': {'id': '5891644015',
    'is_verified': False,
    'prof

In [80]:
# reset de edges handler just in case
edges_handler = []

In [81]:
#extract all comments to an serialized array of comments. 
for req in res_handler:
    for node in req['data']['shortcode_media']['edge_media_to_parent_comment']['edges']:
        edges_handler.append(node['node'])
        
print(f'"edges_handler" has {len(edges_handler)} comments in total.')

"edges_handler" has 1332 comments in total.


In [91]:
# Sample of content
edges_handler[100]

{'id': '18054791563229324',
 'text': 'SO GOOD',
 'created_at': 1589440552,
 'did_report_as_spam': False,
 'owner': {'id': '3583518231',
  'is_verified': False,
  'profile_pic_url': 'https://scontent-gru2-2.cdninstagram.com/v/t51.2885-19/s150x150/131297427_1119461398485362_4851356043381710867_n.jpg?_nc_ht=scontent-gru2-2.cdninstagram.com&_nc_ohc=OVFt9GqE1LQAX9tfL9N&tp=1&oh=bc5ca0745e0b7ee68107cc6d28d5d58c&oe=60078E3D',
  'username': 'herbertgenna'},
 'viewer_has_liked': False,
 'edge_liked_by': {'count': 1},
 'is_restricted_pending': False,
 'edge_threaded_comments': {'count': 0,
  'page_info': {'has_next_page': False, 'end_cursor': None},
  'edges': []}}

In [93]:
result = []

result

[]

In [ ]:
# Starting DFS
for comment in edges_handler:
    result.append([])